# ナイーブベイズ演習　迷惑メール自動振り分け

### データセット
https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection
- ダウンロード
https://archive.ics.uci.edu/ml/machine-learning-databases/00228/
smsspamcollection.zip

In [9]:
import pandas as pd
messages = pd.read_table('smsspamcollection/SMSSpamCollection',
                         names=['label','message'])
messages.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
messages.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
label      5572 non-null object
message    5572 non-null object
dtypes: object(2)
memory usage: 87.1+ KB


### 前処理
ラベルをhamを0に、spamを1のバイナリに変換しましょう。
sklearnを使用して、学習を行う場合、バイナリ(1 or 0)に変換しておく必要があるためです。

In [10]:
messages['label'] = messages['label'].map( {'spam': 1, 'ham': 0} ).astype(int)
print(messages.shape)
messages.head()

(5572, 2)


,label,message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


### Bag of wordsの実装
まずはサンプルから

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
count_vec_sample = CountVectorizer()

In [12]:
sample_messages = ['Thank you for calling.',
            'Thank you for your inquiry',
            'Thanks for keeping in touch.',
            'Thanks for getting in touch with me?']

In [13]:
count_vec_sample.fit(sample_messages)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [14]:
data = count_vec_sample.transform(sample_messages)

In [15]:
data.todense()

matrix([[1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0],
        [0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1],
        [0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0],
        [0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0]])

### Bag of wordsをデータセットに適用

In [20]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

count_vec =  CountVectorizer(analyzer='word', 
                                binary=False, 
                                decode_error='strict',
                                # dtype=<class 'numpy.int64'>, 
                                encoding='utf-8', 
                                input='content',
                                lowercase=True, 
                                max_df=1.0, 
                                max_features=None, 
                                min_df=1,
                                ngram_range=(1, 1), 
                                preprocessor=None, 
                                stop_words=None,
                                strip_accents=None, 
                                token_pattern='(?u)\\b\\w\\w+\\b',
                                tokenizer=None, 
                                vocabulary=None)

In [24]:
count_vec.fit(messages['message'])

#単語毎の出現回数を表示
#count_vec.vocabulary_

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

### データセットを分割する

In [25]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(messages['message'],
                                                    messages['label'],
                                                    random_state=1)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(4179,) (1393,) (4179,) (1393,)


### X_train、X_testをBag of wordsに置き換える

In [26]:
count_vector = CountVectorizer()#fitする前にCountVectorizer()を初期化する。
count_vector.fit(X_train)
X_train = count_vector.transform(X_train)
#count_vector.vocabulary_

In [27]:
X_test = count_vector.transform(X_test)

'''X_testをBags of wordsに変換する際に新しくCountVectorizerクラスのインスタンスを
使用しないのは、X_trainと同じ条件、つまり同じ単語と番号の組み合わせで管理するためです。
新しくfitさせてしまうと、同じ単語でも異なる番号で組み合わされてしまいます。'''

'X_testをBags of wordsに変換する際に新しくCountVectorizerクラスのインスタンスを\n使用しないのは、X_trainと同じ条件、つまり同じ単語と番号の組み合わせで管理するためです。\n新しくfitさせてしまうと、同じ単語でも異なる番号で組み合わされてしまいます。'

### モデル実装・学習
今回のモデルは、MultinomialNBを使用してください。

In [28]:
from sklearn.naive_bayes import MultinomialNB
naive_bayes = MultinomialNB()
naive_bayes.fit(X_train,y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

### 評価
最後に評価を行いましょう。テストデータを使用して、以下の４つの評価を行ってください。
>0. Accuracy
>0. Precision
>0. Recall
>0. F1 Score(f-score)

In [36]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import f1_score, confusion_matrix, classification_report

y_pred = naive_bayes.predict(X_test)

print('Accuracy score: {:.3f}'.format(accuracy_score(y_test,y_pred)))
print('test Precision: {:.3f}'.format(precision_score(y_test, y_pred)))
print('test Recall   : {:.3f}'.format(recall_score(y_test, y_pred)))
print('test F1       : {:.3f}'.format(f1_score(y_test, y_pred)))

report = classification_report(y_test, y_pred)
print(report)

print()
print('**Confusion_Matrix**')
CM = confusion_matrix(y_test, y_pred)
print(CM)
print('[[True_Positive , False_Negative] \n [False_Positive , True_Negative]]')

Accuracy score: 0.989
test Precision: 0.972
test Recall   : 0.941
test F1       : 0.956
             precision    recall  f1-score   support

          0       0.99      1.00      0.99      1208
          1       0.97      0.94      0.96       185

avg / total       0.99      0.99      0.99      1393


**Confusion_Matrix**
[[1203    5]
 [  11  174]]
[[True_Positive , False_Negative] 
 [False_Positive , True_Negative]]


### <font color='Blue'>Precision（適合率）</font>と <font color='Red'>Recall（再現率）</font>について
- 両者はトレードオフ。
- したがって、タスクに応じてどちらを重視するか判断する。

### １. 見逃しが多くても、より正確な予測を求めたいタスクの場合：
- <font color='Blue'>**Precision重視**</font>

    >* スパムメール振り分けタスクなど
      - よりによって重要なメールがスパムと誤まって判定されて開封できなかったらきつい。
       **→ ＜False_Positive(FP)は困る＞**
      - 逆に、スパム判定が見逃されてメールBOXに入ってきたスパムメールがたまにあっても、削除すればいい。
       **→ ＜False_Negative(FN)はまぁまぁ許容する＞**
    >* したがって、多少の見逃しは許容するから、スパム判定時の精度にはこだわりたい。
    <font color='Blue'>***→Precision ＝ TP / (TP + FP) が1に近づくほどうれしい。***</font>
    
### ２. 誤判定が多くても、より見逃しを少なくしたいタスクの場合：
- <font color='Red'>**Recall重視**</font>

    >* 病気の診断、重要機材の部品メンテナンスタスクなど
      - 重大な病気や重要部品の欠陥の判定が見逃されるのはきつい。
        **→ ＜False_Negative(FN)は困る＞**
      - 逆に、見逃しを防ぐあまり的中率がやや下がって病気の誤診があっても、再検査すればいいし、交換した部品が誤判定により実は正常だったとしても事故発生よりはマシ。
          **→ ＜False_Positiveはまぁまぁ許容する＞**
    >* したがって、多少の誤判定は許容するから、見逃さない分類器にこだわりたい。
    <font color='Red'>***→Recall ＝ TP / (TP + FN) が１に近づくほどうれしい。***</font>

### <font color='Green'>F値</font>について
上記のトレードオフを踏まえて、実際に分類器を比較するのに使われるのが<font color='Green'>F値（F-measure)</font>。
>* **PrecisionとRecallの調和平均**
    - <font color='Blue'>Precision : P</font>
    - <font color='Red'>Recall : R</font>
    - <font color='Green'>**F値 = 2 / (1/P)+(1/R)**</font>
    
>* <font color='Blue'>P</font>と<font color='Red'>R</font>のバランスが良ければ<font color='Green'>F値</font>が高くなる。